## Analyzing Dataset

In this notebook, we'll do some preliminary analysis of the data by grouping.

In [8]:
### Loading in libraries.

import pandas as pd
import numpy as np

In [9]:
### In this cell, we load in sales data into a dataframe called 'df'.

df = pd.read_csv("sales_data.csv")
df.head()

,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice
0,52389,1/1/2014,United Kingdom,2152,UK2,Male,11.0,44,10.5,159,0.0,2014,1,159.0
1,52390,1/1/2014,United States,2230,US15,Male,11.5,44-45,11.0,199,0.2,2014,1,159.2
2,52391,1/1/2014,Canada,2160,CAN7,Male,9.5,42-43,9.0,149,0.2,2014,1,119.2
3,52392,1/1/2014,United States,2234,US6,Female,9.5,40,7.5,159,0.0,2014,1,159.0
4,52393,1/1/2014,United Kingdom,2222,UK4,Female,9.0,39-40,7.0,159,0.0,2014,1,159.0


In [10]:
## Let's explore the shape of the dataset.
## Store the shape of the dataset in a variable called 'shape'.
## Find the distribution of males and females. Store the percentage of males in a variable called 'percent_male'.
## Store the average size (UK) in a variable called 'avg_size'.

# your code here
shape = df.shape
num_males = df['Gender'].value_counts()['Male']
percent_male = num_males / len(df)
avg_size = df['Size (UK)'].mean()

In [11]:
## Let's find the average sales in each month.
## Store the average sales per month in a list called 'monthly_average_sales'.

# your code here
monthly_average_sales = df.groupby('Month')['SalePrice'].mean()
monthly_average_sales

Month
1     143.790226
2     142.861640
3     144.919760
4     143.130911
5     144.666213
6     145.582259
7     143.710545
8     143.092032
9     144.388671
10    144.702695
11    143.296794
12    143.312195
Name: SalePrice, dtype: float64

In [12]:
## Let's compare the products sold per shop. 
## Find a 'ProductID' that is sold in the UK but not in Canada. Store it in a variable called 'uk_not_canada'.
## Find a 'ProductID' that is sold in the US but not in UK. Store it in a variable called 'us_not_uk'.
## Which store has the highest average sale price? Store it in a variable called 'highest_price'.


# your code here

df_uk = df[df.Shop == 'UK2']
uk_list = np.unique(df_uk.ProductID)

df_us = df[df.Shop == 'US15']
us_list = np.unique(df_us.ProductID)

df_canada = df[df.Shop == 'CAN7']
canada_list = np.unique(df_canada.ProductID)

for i in us_list:
    if i not in uk_list:
        us_not_uk = i
        break

for i in uk_list:
    if i not in canada_list:
        uk_not_canada = i    
        
np.argmax(df.groupby('Shop')['SalePrice'].mean())
highest_price = df.groupby('Shop')['SalePrice'].mean().index[26]

###  SARIMA Model Predictions
we'll do some further exploration on the dataset, then use a SARIMA model to make predictions.

In [13]:
## Convert the 'Date' column to a pandas timeseries. Make this the index of the dataframe.

# your code here
df.Date = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [14]:
## Implement a SARIMA prediction model to forecast the total amount of sales in the first month of 2017.
## Store the result in a vaiable called 'forecasted_sales'
## Hint: Group by month first and then apply the prediction model.


# your code here
# your code here
df['month_year'] = list(zip(df.Month, df.Year))
sales = df.groupby('month_year').sum()['SalePrice']

from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(endog= sales)
results = model.fit()
forecasted_sales = results.predict(start = len(sales), dynamic= True).iloc[0]

C:\Users\kamal\AppData\Local\Temp\ipykernel_6428\3268010604.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sales = df.groupby('month_year').sum()['SalePrice']
C:\Users\kamal\anaconda3\anaconda-py\envs\data-science\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kamal\anaconda3\anaconda-py\envs\data-science\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\kamal\anaconda3\anaconda-py\envs\data-science\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction res

In [15]:
## Let's analyze some of the statistics of the SARIMA prediction.
## What is the dependent variable? Store the result in a variable called 'dep_variable'.
## How many observations are there? Store the result in a variable called 'num_obs'.
## Finally, what is the coef? Store the result in a variable called 'coef'.

# your code here
results.summary()
dep_variable = 'SalePrice'
num_obs=36
coef = 0.7968